### 计算公式

$WOE_i=log(\frac{G_i/G_{total}}{B_i/B_{total}})$

$IV=\sum (G_i/G_{total} -B_i/B_{total} * ln(\frac{G_i/G_{total}}{B_i/B_{total}} )$

In [1]:
def CalcWOE(df, col, target):
    '''
    :param df: dataframe containing feature and target
    :param col: the feature that needs to be calculated the WOE and iv, usually categorical type
    :param target: good/bad indicator
    :return: WOE and IV in a dictionary
    '''
    total = df.groupby([col])[target].count()
    total = pd.DataFrame({'total': total})
    bad = df.groupby([col])[target].sum()
    bad = pd.DataFrame({'bad': bad})
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
    regroup.reset_index(level=0, inplace=True)
    N = sum(regroup['total'])
    B = sum(regroup['bad'])
    regroup['good'] = regroup['total'] - regroup['bad']
    G = N - B
    regroup['bad_pcnt'] = regroup['bad'].map(lambda x: x*1.0/B)
    regroup['good_pcnt'] = regroup['good'].map(lambda x: x * 1.0 / G)
    regroup['WOE'] = regroup.apply(lambda x: np.log(x.good_pcnt*1.0/x.bad_pcnt),axis = 1)
    WOE_dict = regroup[[col,'WOE']].set_index(col).to_dict(orient='index')
    for k, v in WOE_dict.items():
        WOE_dict[k] = v['WOE']
    IV = regroup.apply(lambda x: (x.good_pcnt-x.bad_pcnt)*np.log(x.good_pcnt*1.0/x.bad_pcnt),axis = 1)
    IV = sum(IV)
    return {"WOE": WOE_dict, 'IV':IV}

In [4]:
def AssignGroup(x, bin):
    N = len(bin)
    #如果值小于最小的分箱值，则取最小的分箱值
    if x<=min(bin):
        return min(bin)
    # 如果值大于最大的分箱值，则取10e10
    elif x>max(bin):
        return 10e10
    else:
        #介于中间的值取又边界值
        for i in range(N-1):
            if bin[i] < x <= bin[i+1]:
                return bin[i+1]

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('application.csv', encoding='ISO-8859-1')

In [7]:
data['y'] = data['loan_status'].map(lambda x: int(x == 'Charged Off'))

In [5]:
#根据之前计算结果直接使用
cutOff=[14400.0, 36000.0]
data['temp2'] = data['annual_inc'].map(lambda x: AssignGroup(x, cutOff))

In [8]:
W=CalcWOE(data,'temp2','y')

In [9]:
W

{'IV': 0.022499080659967422,
 'WOE': {14400.0: -0.6737478488842542,
  36000.0: -0.25078360144745787,
  100000000000.0: 0.0730429907818877}}